In [1]:
from igramscraper.instagram import Instagram
import pandas as pd
import time
import re

In [2]:
#设置代理（ with vpn already)

proxies = {
    'http': 'http://127.0.0.1:1080',
    'https': 'http://127.0.0.1:1080',
}

instagram = Instagram()
instagram.set_proxies(proxies)

# auth登录

In [ ]:
# authentication supported
instagram.with_credentials('用户名', '密码')
instagram.login()

In [4]:
#按账户名获取POST
medias = instagram.get_medias("instantpoteats",9)
if len(medias) == 99999:
    print('OVER THE LIMIT!')
else:
    None
#media = medias[1]

In [7]:
type(medias[0].identifier)

str

# 多media写入
总时间-年月日-时分-content-like-comment-tag-tag_count-at-at_count-media_type-pic_url-post_url

In [6]:
cols = ['post_at','post_date','post_time','content','content_len','pure_content_len','like','comment',
        'festival','tag','tag_count','at','at_count','post_type','video_views','pic_url','post_url','post_id']
df = pd.DataFrame(columns = cols)
df

,post_at,post_date,post_time,content,content_len,pure_content_len,like,comment,festival,tag,tag_count,at,at_count,post_type,video_views,pic_url,post_url,post_id


In [7]:
for i in range(len(medias)):
    #### 总时间-年月日-时分 ####
    post_info = []
    timearray = time.localtime(medias[i].created_time)
    post_info.append(time.strftime("%Y-%m-%d %H:%M:%S", timearray))
    post_info.append(time.strftime("%Y-%m-%d", timearray))
    post_info.append(time.strftime("%H:%M:%S", timearray))       #which fucking timezone?


    ### content-like-comment  ###
    po_content = medias[i].caption
    post_info.append(po_content)
    post_info.append(len(po_content))
    
    ### calculate pure content len
    ### pre-extract tag-at and cal len
    
    ### tag-tag_count-at-at_count ###
    tag = re.findall(r'(#[\w]+)',str(po_content))
    tag_num = len(tag)
    if tag_num != 0:
        tag_word_len = 0
        for t_i in range(tag_num):
            tag_word_len += (len(tag[t_i]) + 1)
        tag = ', '.join(tag)
    else:
        tag = 'Null'
        tag_word_len = 0

    
    
    
    ### @选取
    at_user = re.findall(r'(@[\w]+)',str(po_content))
    at_user_num = len(at_user)
    if at_user_num != 0:
        at_word_len = 0
        for a_i in range(at_user_num):
            at_word_len += len(at_user[a_i])
        at_user = ', '.join(at_user)
    else:
        at_user = 'Null'
        at_word_len = 0
    
    pure_content_len = len(po_content) - tag_word_len - at_word_len
    
    post_info.append(pure_content_len)
    post_info.append(medias[i].likes_count)
    post_info.append(medias[i].comments_count)


    ### festival extract
    fes = re.findall(r'([\S]+s [D|d][A|a][Y|y])',str(po_content))
    fes_num = len(fes)
    if fes_num != 0:
        fes = ', '.join(fes)
    else:
        fes = 'Null'
    post_info.append(fes)
    
    post_info.append(tag)
    post_info.append(tag_num)
    post_info.append(at_user)
    post_info.append(at_user_num)
    post_info.append(medias[i].type)
    if medias[i].type == "video":
        post_info.append(medias[i].video_view_count)
    else:
        post_info.append("Null")
    post_info.append(medias[i].image_high_resolution_url)
    post_info.append(medias[i].link)
    post_info.append(medias[i].identifier)
    #print(post_info)
    x = pd.Series(post_info,index=df.columns)
    df = df.append(x,ignore_index = True)

In [8]:
df.head()

,post_at,post_date,post_time,content,content_len,pure_content_len,like,comment,festival,tag,tag_count,at,at_count,post_type,video_views,pic_url,post_url,post_id
0,2020-09-04 19:01:12,2020-09-04,19:01:12,If you’ve never had horchata you are in for a ...,1009,620,248,3,Null,"#instantpoteats, #instantpot, #instantpotcooki...",28,"@geekypoet, @instantpoteats",2,image,Null,https://scontent-lax3-1.cdninstagram.com/v/t51...,https://www.instagram.com/p/CEtlAIbHr-s,2390729713969446828
1,2020-09-04 05:01:10,2020-09-04,05:01:10,These Instant Pot creamed beets make a perfect...,913,574,164,3,Null,"#instantpoteats, #instantpot, #instantpotcooki...",24,@instantpoteats,1,image,Null,https://scontent-lax3-1.cdninstagram.com/v/t51...,https://www.instagram.com/p/CEsE3WgAM3a,2390306898143399386
2,2020-09-03 19:01:09,2020-09-03,19:01:09,"Easy, creamy Instant Pot Lentil Curry. Amazing...",822,465,307,3,Null,"#instantpoteats, #instantpot, #instantpotcooki...",25,"@wellplated, @instantpoteats",2,image,Null,https://scontent-lax3-1.cdninstagram.com/v/t51...,https://www.instagram.com/p/CErANUFgddL,2390004942170609483
3,2020-09-03 05:01:10,2020-09-03,05:01:10,We love using beetroot in cooking because it’s...,861,495,216,3,Null,"#instantpoteats, #instantpot, #instantpotcooki...",26,@instantpoteats,1,image,Null,https://scontent-lax3-1.cdninstagram.com/v/t51...,https://www.instagram.com/p/CEpgEYOpkl-,2389582115678603646
4,2020-09-02 19:01:05,2020-09-02,19:01:05,Hearty and healthy veggie-packed vegan corn ch...,831,465,594,7,Null,"#instantpoteats, #instantpot, #instantpotcooki...",24,"@theroastedroot, @instantpoteats",2,image,Null,https://scontent-lax3-1.cdninstagram.com/v/t51...,https://www.instagram.com/p/CEobZcqH2Bh,2389280098334236769


In [9]:
df.to_excel("instantpoteats.xlsx", index = False)